In [17]:
# Import Python Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats.power as smp

# Question 1

A. How large a sample size would be needed to detect an increase in click throughs equal to 10% of the mean, with 90% power and 5% significance? Assume the mean click through rate is 5%.

In [18]:
# Question 1A: The required sample size
std_size = 0.1 * 0.05 / np.sqrt(0.05 * (1 - 0.05))
sample_size_required = smp.tt_ind_solve_power(alpha = 0.05, power = 0.9,
                                              effect_size = std_size)
print('The required sample size would be:', sample_size_required)

The required sample size would be: 39929.15521524072


B. If the typical treatment arm has 3200 observations, what is the minimum detectable effect (not standardized), given 5% significance?

In [19]:
# Question 1B: The minimum detectable effect
mde_std = smp.tt_ind_solve_power(alpha = 0.05, power = 0.9, nobs1 = 3200)
std_treat = np.sqrt(0.05 * (1 - 0.05)) # 5% click through rate
mde = mde_std * std_treat
print('The minimum detectable effect (not standardized) is:', mde)

The minimum detectable effect (not standardized) is: 0.017664008734157317


C. If the typical treatment arm has 3200 observations, what is our power to detect an effect equal to 10% of the mean, with 5% significance?

In [20]:
# Question 1C: Power calculations
power_typical = smp.tt_ind_solve_power(alpha = 0.05, nobs1 = 3200, effect_size = std_size)
print('The power to detect is:', power_typical)

The power to detect is: 0.1506084136476183


D. Your organization has a website which gets 4,000 views/hour. You are considering running an A/B test of headlines to determine headline effectiveness. For how long would you have to run the test to be able to detect an increase in the click through rate from 5% to 5.5% (with 90% power and 5% significance)?  ("A/B test" means you have 1 control arm and 1 treatment arm.)

In [21]:
# Question 1D: Required hours
new_std_size = (0.055 - 0.05) / np.sqrt(0.05 * (1 - 0.05))
nobs = smp.tt_ind_solve_power(effect_size = new_std_size, alpha=0.05, power=0.9)
views_per_hour = 4000
hours_required = nobs / (views_per_hour / 2)
print('Required hours to detect the increase in click through rate is:', hours_required)

Required hours to detect the increase in click through rate is: 19.96457760713906


In an A/B test, the total traffic is split evenly between the two groups: the control arm and the treatment arm.
This means that out of the total views_per_hour (4,000 in this case), only half (2,000) will go to each arm.

# Question 2

Calculate the total number of impression and total number of clicks in each arm of the experiment, and then calculate the overall clickthrough rate in each arm.

A. What is the difference in means?


In [22]:
# Clickability_test_id: Identifier for different experiments
# Impressions0: impressions (number of headline views) for the no-question-mark condition
# Impressions1: impression (number of headline views) for the question-mark condition
# clicks0: number of clicks for the no-question-mark condition
# clicks1: number of clicks for the question-mark condition
data = pd.read_csv('/content/upworthy_questions.csv')
data.head()

clickability_test_id  impressions0  impressions1  clicks0  clicks1
0  51436060220cb800020001e3          2707          2606       50       42
1  51436061220cb800020001e8          2702          2720       21       14
2  51436061220cb800020001ec          2817          2945       22       29
3  51436063220cb800020002a7          5268          5414      112      120
4  51436063220cb800020002cc          5293          5377       58       74

In [23]:
# Question 2A: Difference in means
impressions0_total = data['impressions0'].sum()
impressions1_total = data['impressions1'].sum()
clicks0_total = data['clicks0'].sum()
clicks1_total = data['clicks1'].sum()

''' Difference if swap impressions and clicks???'''
# control_grp = impressions0_total / clicks0_total
# treat_grp = impressions1_total / clicks1_total

control_grp = clicks0_total / impressions0_total
treat_grp = clicks1_total / impressions1_total

diff_means = treat_grp - control_grp
print('Difference in means is:', diff_means)

Difference in means is: -0.0011409824019145802


B. Form a 95% confidence interval for the difference in means. Is the difference statistically significant?

In [24]:
# Question 2B: Statistically significant?
std_control = np.sqrt(control_grp * (1 - control_grp))
std_treatment = np.sqrt(treat_grp * (1 - treat_grp))

sem_control = std_control / np.sqrt(impressions0_total)
sem_treatment = std_treatment / np.sqrt(impressions1_total)

ate = treat_grp - control_grp
se_ate = np.sqrt(sem_control**2 + sem_treatment**2)

ate_lb = ate - 1.96 * se_ate
ate_ub = ate + 1.96 * se_ate
print('ATE equals to:', ate)
print('95% CI for ATE:', (ate_lb, ate_ub))

ATE equals to: -0.0011409824019145802
95% CI for ATE: (-0.0012004499012693817, -0.0010815149025597787)


It is statistically significant.

# Question 3

Now treat each experiment as a separate observation. For each experiment, calculate the effect of question mark on the outcome. This will give you 6,939 ATEs.

A. What is the average ATE, averaging over all the experiments?

In [25]:
# Question 3A: Average ATE
data['ate'] = (data['clicks1'] / data['impressions1']) - (data['clicks0'] / data['impressions0'])
data['std_ctrl'] = np.sqrt(data['clicks0'] / data['impressions0'] * (1 - data['clicks0'] / data['impressions0']))
data['std_treat'] = np.sqrt(data['clicks1'] / data['impressions1'] * (1 - data['clicks1'] / data['impressions1']))

data['se'] = np.sqrt((data['std_ctrl'] / np.sqrt(data['impressions0'])) ** 2 + (data['std_treat'] / np.sqrt(data['impressions1'])) **2)

data['ate_lb'] = data['ate'] - 1.96 * data['se']
data['ate_ub'] = data['ate'] + 1.96 * data['se']

mean_ate = data['ate'].mean()
print('The average ATE is ', mean_ate, 'averaging over all the experiments.')

The average ATE is  -0.0012118772185663103 averaging over all the experiments.


B. How many of the ATEs are statistically significant? (hint: consider making a confidence interval for each experiment)

In [26]:
# Abbreviated version
# B. How many of the ATEs are statistically significant?
subdata = data[(data['ate_lb'] > 0) | (data['ate_ub'] < 0)]
count_subdata = subdata['ate'].count()
print('Number of statistically significant ATEs:', count_subdata)

Number of statistically significant ATEs: 2315


C. What is the mean ATE among experiments where the ATE is statistically significant?

In [27]:
# C. The mean ATE among experiments where the ATE is statistically significant.
mean_ate_adj = subdata['ate'].mean()
print('Mean of statistically significant ATEs:', mean_ate_adj)

Mean of statistically significant ATEs: -0.0030497218997479945


D. Why is the answer in (c) different than in (b)?

In [28]:
'''in Part A, the average ATE is calculated using the entire dataset,
whereas in Part C, the average ATE is derived solely from statistically
significant ATEs.'''
'''When researchers or analysts filter out results based on statistical
significance, they inherently select ATEs that are further from zero
(more extreme), because more moderate or smaller effects are less likely to meet
the threshold for significance. This selection makes the significant ATEs appear
larger, even if the true underlying effects are more moderate.'''

'When researchers or analysts filter out results based on statistical\nsignificance, they inherently select ATEs that are further from zero\n(more extreme), because more moderate or smaller effects are less likely to meet\nthe threshold for significance. This selection makes the significant ATEs appear\nlarger, even if the true underlying effects are more moderate.'

E. If you were an organization that ran many experiments and only paid attention to the significant ones, would your answers be reliable? Explain briefly.

In [29]:
'''No. Because Focusing only on statistically significant outcomes can lead to
"publication bias" or "selective reporting," where non-significant results are
ignored, skewing the overall findings. This approach inflates the perceived
effectiveness or impact of interventions, as it overlooks cases where the effect
may be weaker or inconsistent, thus misrepresenting the true results of the
experiments.'''
'''p-haking selectively reporting findings to achieve statistically significant
results, often with a p-value below 0.05, which is a common threshold for
statistical significance. This can lead to misleading or exaggerated results in
scientific research, as the findings may not truly reflect reality but rather
the result of excessive tweaking.'''

'p-haking selectively reporting findings to achieve statistically significant\nresults, often with a p-value below 0.05, which is a common threshold for\nstatistical significance. This can lead to misleading or exaggerated results in\nscientific research, as the findings may not truly reflect reality but rather\nthe result of excessive tweaking.'

# Question 4

Develop a hypothesis about what features of headline generate higher click through rates. Think about an experiment that would let you test it, using the Upworthy experiment database. The key challenge in testing your hypothesis is whether there are experiments that A/B test the feature, and whether you can identify those experiments. For example, we can tell if there is a question in a headline because the data have a headline column, and we can see if it contains a ‘?’. Make a pre-analysis plan at aspredicted.org, and turn in your pre-analysis plan with your write-up.

**Comments**

Overall, this is a thoughtful and well-structured plan. You might consider minor adjustments, such as addressing outlier handling